# Proyecto Final - Diego Acosta

In [38]:
# Cell 1: Imports
import math
from dataclasses import dataclass
from typing import Dict, List, Optional

import ipywidgets as widgets
from IPython.display import display, clear_output

import matplotlib.pyplot as plt

In [39]:
# Cell 2: Model — Nodo, Grafo con aristas manuales y Dijkstra
@dataclass
class Node:
    name: str
    lat: float
    lon: float

class Graph:
    def __init__(self):
        self.nodes: Dict[str, Node] = {}
        self.adj: Dict[str, Dict[str, float]] = {}

    def add_node(self, name: str, lat: float, lon: float):
        if name in self.nodes:
            raise ValueError(f"Node '{name}' already exists.")
        self.nodes[name] = Node(name, lat, lon)
        self.adj[name] = {}

    def update_node(self, name: str, lat: float, lon: float):
        if name not in self.nodes:
            raise KeyError(f"Node '{name}' not found.")
        self.nodes[name].lat, self.nodes[name].lon = lat, lon

    def delete_node(self, name: str):
        if name not in self.nodes:
            raise KeyError(f"Node '{name}' not found.")
        del self.nodes[name]
        del self.adj[name]
        for nbrs in self.adj.values():
            nbrs.pop(name, None)

    def add_edge(self, u: str, v: str, dist: float):
        if u not in self.nodes or v not in self.nodes:
            raise KeyError("Both nodes must exist.")
        self.adj[u][v] = dist
        self.adj[v][u] = dist

    def list_nodes(self) -> List[str]:
        return list(self.nodes.keys())

    def list_edges(self) -> Dict[str, Dict[str, float]]:
        return self.adj

    def _dijkstra(self, start: str):
        dist = {n: math.inf for n in self.nodes}
        prev = {n: None      for n in self.nodes}
        dist[start] = 0
        unvis = set(self.nodes)
        while unvis:
            u = min(unvis, key=lambda x: dist[x])
            unvis.remove(u)
            for v, w in self.adj[u].items():
                if v in unvis and dist[u] + w < dist[v]:
                    dist[v] = dist[u] + w
                    prev[v] = u
        return dist, prev

    def shortest_path(self, start: str, end: str) -> (Optional[List[str]], float):
        if start not in self.nodes or end not in self.nodes:
            raise KeyError("Start or end node not found.")
        dist, prev = self._dijkstra(start)
        if dist[end] == math.inf:
            return None, math.inf
        path = []
        u = end
        while u:
            path.insert(0, u)
            u = prev[u]
        return path, dist[end]

In [40]:
# Cell 3: Instantiate Graph and preload 10 sample nodes and edges

import math

# A standalone haversine function, since Graph no longer defines _haversine
def haversine(lat1, lon1, lat2, lon2) -> float:
    R = 6371.0  # Earth radius in km
    φ1, φ2 = math.radians(lat1), math.radians(lat2)
    dφ = math.radians(lat2 - lat1)
    dλ = math.radians(lon2 - lon1)
    a = math.sin(dφ/2)**2 + math.cos(φ1)*math.cos(φ2)*math.sin(dλ/2)**2
    return 2 * R * math.atan2(math.sqrt(a), math.sqrt(1-a))

# 1) Create the graph
g = Graph()

# 2) Preload 10 sample nodes: (name, latitude, longitude)
initial_nodes = [
    ("A",  0.0,  0.0),
    ("B",  1.0,  2.0),
    ("C",  3.0,  1.0),
    ("D", -1.0,  2.5),
    ("E",  2.0, -2.0),
    ("F", -2.0, -1.0),
    ("G",  4.0,  0.0),
    ("H",  0.0, -3.0),
    ("I",  5.0,  3.0),
    ("J", -3.0, -2.0),
]
for name, lat, lon in initial_nodes:
    g.add_node(name, lat, lon)

# 3) Preload 10 sample undirected edges, weighting by true geodesic distance
initial_edges = [
    ("A", "B"),
    ("A", "C"),
    ("B", "D"),
    ("C", "E"),
    ("D", "F"),
    ("E", "G"),
    ("F", "H"),
    ("G", "I"),
    ("H", "J"),
    ("I", "J"),
]
for u, v in initial_edges:
    d = haversine(
        g.nodes[u].lat, g.nodes[u].lon,
        g.nodes[v].lat, g.nodes[v].lon
    )
    g.add_edge(u, v, d)

# 4) If your UI cell defines `refresh_all()`, call it to populate dropdowns immediately
try:
    refresh_all()
except NameError:
    pass

In [ ]:
# Cell 4: Interfaz — CRUD nodos, aristas, ruta más corta y visualización

# — Widgets nodos —
name_in   = widgets.Text(description="Name")
lat_in    = widgets.FloatText(description="Lat.")
lon_in    = widgets.FloatText(description="Lon.")
btn_add   = widgets.Button(description="Add Node")
btn_upd   = widgets.Button(description="Upd Node")
btn_del   = widgets.Button(description="Del Node")
btn_lst   = widgets.Button(description="List Nodes")
out_nodes = widgets.Output()

# — Widgets aristas —
edge_u    = widgets.Dropdown(options=[], description="From")
edge_v    = widgets.Dropdown(options=[], description="To")
edge_d    = widgets.FloatText(description="Dist.")
btn_edge  = widgets.Button(description="Add Edge")
btn_lst_e = widgets.Button(description="List Edges")
out_edges = widgets.Output()

# — Widgets ruta más corta —
start_dd  = widgets.Dropdown(options=[], description="Start")
end_dd    = widgets.Dropdown(options=[], description="End")
btn_path  = widgets.Button(description="Compute Path")
out_path  = widgets.Output()

# — Widget visualización —
btn_show  = widgets.Button(description="Show Graph")
out_show  = widgets.Output()

# — Función para refrescar dropdowns de nodos en toda la UI —
def refresh_all():
    opts = g.list_nodes()
    edge_u.options = opts
    edge_v.options = opts
    start_dd.options = opts
    end_dd.options   = opts

# — Handlers nodos —
def on_add_node(b):
    with out_nodes:
        clear_output()
        try:
            g.add_node(name_in.value, lat_in.value, lon_in.value)
            print(f"Added node '{name_in.value}'")
        except Exception as e:
            print("Error:", e)
        refresh_all()

def on_upd_node(b):
    with out_nodes:
        clear_output()
        try:
            g.update_node(name_in.value, lat_in.value, lon_in.value)
            print(f"Updated node '{name_in.value}'")
        except Exception as e:
            print("Error:", e)
        refresh_all()

def on_del_node(b):
    with out_nodes:
        clear_output()
        try:
            g.delete_node(name_in.value)
            print(f"Deleted node '{name_in.value}'")
        except Exception as e:
            print("Error:", e)
        refresh_all()

def on_lst_node(b):
    with out_nodes:
        clear_output()
        for n in g.list_nodes():
            nd = g.nodes[n]
            print(f"{n}: ({nd.lat:.6f}, {nd.lon:.6f})")

# — Handlers aristas —
def on_add_edge(b):
    with out_edges:
        clear_output()
        try:
            g.add_edge(edge_u.value, edge_v.value, edge_d.value)
            print(f"Edge {edge_u.value} ↔ {edge_v.value} = {edge_d.value}")
        except Exception as e:
            print("Error:", e)

def on_lst_edge(b):
    with out_edges:
        clear_output()
        for u, nbrs in g.list_edges().items():
            line = ", ".join(f"{v}({d:.2f})" for v,d in nbrs.items())
            print(f"{u} → {line}")

# — Handler camino más corto —
def on_path(b):
    with out_path:
        clear_output()
        try:
            path, dist = g.shortest_path(start_dd.value, end_dd.value)
            if path is None:
                print("No path found.")
            else:
                print("Path:", " → ".join(path))
                print(f"Distance: {dist:.2f} km")
        except Exception as e:
            print("Error:", e)

# — Handler mostrar grafo —
def on_show(b):
    with out_show:
        clear_output()
        # Plot
        fig, ax = plt.subplots()
        # dibuja nodos
        for n, nd in g.nodes.items():
            ax.scatter(nd.lon, nd.lat)
            ax.text(nd.lon, nd.lat, n)
        # dibuja aristas (sin duplicar)
        seen = set()
        for u, nbrs in g.adj.items():
            for v in nbrs:
                if (v,u) in seen: continue
                seen.add((u,v))
                x0,y0 = g.nodes[u].lon, g.nodes[u].lat
                x1,y1 = g.nodes[v].lon, g.nodes[v].lat
                ax.plot([x0,x1],[y0,y1])
        ax.set_xlabel("Longitude")
        ax.set_ylabel("Latitude")
        plt.show()

# — Conecta eventos —
btn_add .on_click(on_add_node)
btn_upd .on_click(on_upd_node)
btn_del .on_click(on_del_node)
btn_lst .on_click(on_lst_node)
btn_edge.on_click(on_add_edge)
btn_lst_e.on_click(on_lst_edge)
btn_path.on_click(on_path)
btn_show.on_click(on_show)

# — Inicializa y muestra UI —
refresh_all()
display(widgets.VBox([
    widgets.Label("=== Nodes ==="),
    widgets.HBox([name_in, lat_in, lon_in]),
    widgets.HBox([btn_add, btn_upd, btn_del, btn_lst]),
    out_nodes,
    widgets.Label("=== Edges ==="),
    widgets.HBox([edge_u, edge_v, edge_d]),
    widgets.HBox([btn_edge, btn_lst_e]),
    out_edges,
    widgets.Label("=== Shortest Path ==="),
    widgets.HBox([start_dd, end_dd, btn_path]),
    out_path,
    widgets.Label("=== Graph Visualization ==="),
    btn_show, out_show
]))